# Análisis de dataset

## Importar librerías

In [1]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np
# ....

## Carga y Preprocesamiento de Datos

### Carga del dataset

### Exploración inicial del dataset

### Preprocesar Datos

## Análisis exploratorio de datos


## Selección de modelo usando técnicas de aprendizaje supervisado


### Selección de características


### Validación de balanceo de clases


### Selección de modelo supervisado


### Ajuste de hiperparámetros del modelo


### Evaluación del modelo seleccionado


## Análisis de clúster con aprendizaje no supervisado


### PCA


### Clustering
